In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import ast
from tqdm import tqdm

tqdm.pandas()
df = pd.read_csv('/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/tokens_split/eadily_tokens_6')
df = df.rename(columns={'stopwords': 'tagged'})
df.head(10)
df['tagged'] = df['tagged'].progress_apply(ast.literal_eval)

100%|██████████| 670/670 [00:02<00:00, 332.74it/s]


In [5]:
# tagged 컬럼 삭제 버전
from collections import Counter
from tqdm import tqdm

ALLOWED_TAGS = ['NNG', 'VA', 'VAX', 'MAG', 'negations']
# 순서를 고려하지는 않아도 됨 / n-gram의 빈도수가 중요한 거니까
def generate_ngrams(token_list, max_n=5, allowed_tags=ALLOWED_TAGS):
    words = [token[0] for token in token_list if token[1] in allowed_tags]
    ngram_counter = Counter()
    for n in range(1, max_n + 1):
        ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
        ngram_counter.update(ngrams)
    return ngram_counter

def sorted_ngrams(token_list, max_n=5):
    counter = generate_ngrams(token_list, max_n)
    sorted_ng = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    return str(sorted_ng)

tqdm.pandas()
# 각 행별로 n-그램 결과를 'ngram' 컬럼에 추가
df['ngram'] = df['tagged'].progress_apply(lambda x: sorted_ngrams(x, max_n=5))
df[['date', 'ngram']].to_csv('/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/ngram_token6_results.csv', index=False)

100%|██████████| 670/670 [00:01<00:00, 435.71it/s]


In [10]:
pd.read_csv('ngram_bond_results.csv')['ngram'][0]

FileNotFoundError: [Errno 2] No such file or directory: 'ngram_bond_results.csv'

In [7]:
# labeling
# ngram 파일에서 다시 리스트를 뽑아옴
import pandas as pd
import ast

df_ngram = pd.read_csv('/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/ngram_token6_results.csv')
df_call = pd.read_csv('/content/drive/MyDrive/BOK-TEAM-2/데이터 수집/콜금리 데이터/call_rate_data.csv')
# 안 헷갈리게 이름 통일
df_call = df_call.rename(columns={'시점': 'date'})
df_call = df_call.rename(columns= {'값':'rate'})
# pandas 통해 날짜 형식으로 바꿔주기
df_call['date'] = pd.to_datetime(df_call['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df_call = df_call.sort_values(by='date', ascending=True) # 이미 오름차순으로 정리되어 있지만 오류 방지 차
df_ngram['date'] = pd.to_datetime(df_ngram['date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
print(df_ngram['date'].head(10))

df_callrate = pd.DataFrame(df_call[['date', 'rate']])
df_callrate.head(10)



0    2024-11-21
1    2024-11-21
2    2024-11-21
3    2024-11-21
4    2024-11-21
5    2024-11-21
6    2024-11-21
7    2024-11-21
8    2024-11-21
9    2024-11-21
Name: date, dtype: object


,date,rate
0,2008-01-01,5.0
1,2008-01-02,5.0
2,2008-01-03,5.0
3,2008-01-04,5.0
4,2008-01-05,5.0
5,2008-01-06,5.0
6,2008-01-07,5.0
7,2008-01-08,5.0
8,2008-01-09,5.0
9,2008-01-10,5.0


In [9]:
import datetime
for idx in range(len(df_ngram)):
    d = df_ngram['date'][idx] # 오늘 날짜
    # 오늘 날짜의 금리
    today_rate = df_callrate[df_callrate['date'] == d]['rate'].values[0]

    # 30일 후 날짜 계산
    d_dt = datetime.datetime.strptime(d, "%Y-%m-%d")
    d_later = (d_dt + datetime.timedelta(days=30)).strftime("%Y-%m-%d")

    # 30일 뒤 금리
    later_rate = df_callrate[df_callrate['date'] == d_later]['rate'].values[0]

    # 새로운 ngram 리스트 생성
    updated_tokens = []
    for token in ast.literal_eval(df_ngram['ngram'][idx]):  # 문자열을 리스트로 변환
        if today_rate > later_rate:
            updated_tokens.append(token + (-1,))
        elif today_rate < later_rate:
            updated_tokens.append(token + (1,))
        else:
            updated_tokens.append(token + (0,))

    # 변환된 ngram을 다시 할당
    df_ngram.at[idx, 'ngram'] = updated_tokens

    # 첫 번째 데이터만 확인용 출력
    print(df_ngram['date'][0])
    break  # 첫 번째 행만 테스트하고 종료

2024-11-21


In [10]:
df_ngram['ngram'][0]

[(('금리',), 12, -1),
 (('국채선물',), 6, -1),
 (('하락',), 5, -1),
 (('국고채',), 4, -1),
 (('시장',), 4, -1),
 (('기록',), 4, -1),
 (('계약',), 4, -1),
 (('내외',), 3, -1),
 (('거래일',), 3, -1),
 (('인하',), 3, -1),
 (('내외', '하락'), 3, -1),
 (('강보합',), 2, -1),
 (('출발',), 2, -1),
 (('국내',), 2, -1),
 (('가격',), 2, -1),
 (('엠피',), 2, -1),
 (('닥터',), 2, -1),
 (('기준',), 2, -1),
 (('대비',), 2, -1),
 (('틱',), 2, -1),
 (('외국인',), 2, -1),
 (('순매수',), 2, -1),
 (('순매도',), 2, -1),
 (('중립',), 2, -1),
 (('미국',), 2, -1),
 (('가능성',), 2, -1),
 (('하락', '강보합'), 2, -1),
 (('강보합', '출발'), 2, -1),
 (('금리', '내외'), 2, -1),
 (('엠피', '닥터'), 2, -1),
 (('거래일', '대비'), 2, -1),
 (('국채선물', '외국인'), 2, -1),
 (('외국인', '계약'), 2, -1),
 (('계약', '순매수'), 2, -1),
 (('계약', '순매도'), 2, -1),
 (('인하', '가능성'), 2, -1),
 (('내외', '하락', '강보합'), 2, -1),
 (('하락', '강보합', '출발'), 2, -1),
 (('금리', '내외', '하락'), 2, -1),
 (('국채선물', '외국인', '계약'), 2, -1),
 (('내외', '하락', '강보합', '출발'), 2, -1),
 (('채권',), 1, -1),
 (('반대',), 1, -1),
 (('분봉',), 1, -1),
 (('차트',), 1, -1),
 (('

## 최종 코드
- 이데일리 파일 반복


In [ ]:
import pandas as pd
import ast
from tqdm import tqdm
from collections import Counter
import datetime

# tqdm 설정
tqdm.pandas()

# 사용할 품사 태그
ALLOWED_TAGS = ['NNG', 'VA', 'VAX', 'MAG', 'negations']

# ngram 생성 함수
def generate_ngrams(token_list, max_n=5, allowed_tags=ALLOWED_TAGS):
    words = [token[0] for token in token_list if token[1] in allowed_tags]
    ngram_counter = Counter()
    for n in range(1, max_n + 1):
        ngrams = [tuple(words[i:i+n]) for i in range(len(words) - n + 1)]
        ngram_counter.update(ngrams)
    return ngram_counter
# ngram 정렬 후 문자열로 변환
def sorted_ngrams(token_list, max_n=5):
    counter = generate_ngrams(token_list, max_n)
    sorted_ng = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    return str(sorted_ng)

# 이데일리 파일 6개
for i in range(1, 7):
    # 토큰 파일 로드
    df = pd.read_csv(f'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/tokens_split/eadily_tokens_{i}')
    df = df.rename(columns={'stopwords': 'tagged'})
    df['tagged'] = df['tagged'].progress_apply(ast.literal_eval)
    print(f'[{i}] Token file loaded')

    # ngram 시행
    # 각 행별로 n-그램 결과를 'ngram' 컬럼에 추가
    df['ngram'] = df['tagged'].progress_apply(lambda x: sorted_ngrams(x, max_n=5))
    df[['date', 'ngram']].to_csv(
        f'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/tokens_split/ngram_token{i}_results.csv',
        encoding='utf-8', index=False
    )
    print(f'[{i}] N-gram completed')

    # 콜금리 데이터 & 엔그램 데이터 프레임 정리
    df_ngram = pd.read_csv(f'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/tokens_split/ngram_token{i}_results.csv')
    df_call = pd.read_csv('/content/drive/MyDrive/BOK-TEAM-2/데이터 수집/콜금리 데이터/call_rate_data.csv')

    # 안 헷갈리게 이름 통일
    df_call = df_call.rename(columns={'시점': 'date', '값': 'rate'})

    # pandas 통해 날짜 형식으로 바꿔주기
    df_call['date'] = pd.to_datetime(df_call['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
    df_call = df_call.sort_values(by='date', ascending=True)

    df_ngram['date'] = pd.to_datetime(df_ngram['date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

    # date를 인덱스로 설정하여 조회 성능 향상
    df_callrate = df_call.set_index('date')
    print(f'[{i}] Data sorted')

    # 라벨링 튜플 추가
    def label_ngram(row):
        d = row['date']
        try:
            today_rate = df_callrate.loc[d, 'rate']
            d_later = (datetime.datetime.strptime(d, "%Y-%m-%d") + datetime.timedelta(days=30)).strftime("%Y-%m-%d")
            later_rate = df_callrate.loc[d_later, 'rate']

            updated_tokens = []
            for token in ast.literal_eval(row['ngram']):
                if today_rate > later_rate:
                    updated_tokens.append(token + (-1,))
                elif today_rate < later_rate:
                    updated_tokens.append(token + (1,))
                else:
                    updated_tokens.append(token + (0,))
            return updated_tokens
        except KeyError:
            return row['ngram']  # 금리 데이터가 없을 경우 기존 값 유지

    df_ngram['ngram'] = df_ngram.progress_apply(label_ngram, axis=1)
    # 최종 파일 저장
    df_ngram.to_csv(
        f'/content/drive/MyDrive/BOK-TEAM-2/데이터 전처리/이데일리/tokens_split/ngram_token{i}_results.csv',
        encoding='utf-8', index=False
    )
    print(f'[{i}] File saved')

100%|██████████| 20000/20000 [01:02<00:00, 319.36it/s]


[1] Token file loaded


100%|██████████| 20000/20000 [00:33<00:00, 602.07it/s]


[1] N-gram completed
[1] Data sorted


100%|██████████| 20000/20000 [23:21<00:00, 14.27it/s]


...

100%|██████████| 20000/20000 [00:31<00:00, 637.62it/s]


[2] N-gram completed
[2] Data sorted


100%|██████████| 20000/20000 [25:42<00:00, 12.96it/s]


[2] File saved


 10%|▉         | 1931/20000 [00:13<01:00, 298.39it/s]